### claude-sonnet-4 / edited by author

## loop 的本质

**事件循环本质上是一个无限循环**，它不断地：
1. 检查是否有待处理的任务
2. 执行这些任务
3. 处理 I/O 事件
4. 重复这个过程

## loop 的内部结构

In [ ]:
import asyncio

# 创建事件循环
loop = asyncio.new_event_loop()
print(type(loop))  # <class 'asyncio.unix_events._UnixSelectorEventLoop'>

# 事件循环的关键组件
print(f"是否运行中: {loop.is_running()}")
print(f"是否已关闭: {loop.is_closed()}")


## loop 的核心功能

### 1. 任务调度器
```python
import asyncio

async def demo_task(name, delay):
    print(f"{name} 开始")
    await asyncio.sleep(delay)
    print(f"{name} 结束")

# 事件循环管理多个任务
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)

# 创建任务
task1 = loop.create_task(demo_task("任务1", 1))
task2 = loop.create_task(demo_task("任务2", 2))

# 运行直到所有任务完成
loop.run_until_complete(asyncio.gather(task1, task2))
```

### 2. I/O 多路复用器
```python
import asyncio
import socket

async def handle_client(reader, writer):
    data = await reader.read(100)
    print(f"收到数据: {data.decode()}")
    writer.write(b"HTTP/1.1 200 OK\r\n\r\nHello World")
    await writer.drain()
    writer.close()

# 事件循环处理网络 I/O
loop = asyncio.new_event_loop()
server = loop.run_until_complete(
    asyncio.start_server(handle_client, 'localhost', 8888)
)
```

### 3. 定时器管理
```python
import asyncio

def callback():
    print("定时器触发")

loop = asyncio.new_event_loop()
# 安排在 2 秒后执行
loop.call_later(2, callback)
# 在指定时间执行
loop.call_at(loop.time() + 3, callback)
```

## loop 的生命周期

In [ ]:
# 这个程序要单独跑，否则error

import asyncio

# 1. 创建事件循环
loop = asyncio.new_event_loop()
print(f"创建后状态: running={loop.is_running()}, closed={loop.is_closed()}")

# 2. 设置为当前线程的事件循环
asyncio.set_event_loop(loop)

# 3. 运行事件循环
async def main():
    print("在事件循环中运行")
    print(f"运行时: running={loop.is_running()}")
    await asyncio.sleep(1)

# 运行时状态
print(f"运行前: running={loop.is_running()}")
loop.run_until_complete(main())
print(f"运行后: running={loop.is_running()}")

# 4. 关闭事件循环
loop.close()
print(f"关闭后: closed={loop.is_closed()}")

## loop 的内部数据结构

事件循环内部包含：

```python
# 简化的事件循环结构
class EventLoop:
    def __init__(self):
        self._ready = collections.deque()      # 就绪任务队列
        self._scheduled = []                   # 定时任务堆
        self._selector = selectors.DefaultSelector()  # I/O 选择器
        self._running = False
        self._closed = False
    
    def run_forever(self):
        self._running = True
        while self._running:
            # 处理就绪任务
            self._run_ready_tasks()
            
            # 处理 I/O 事件
            self._run_io_events()
            
            # 处理定时器
            self._run_scheduled_tasks()
    
    def create_task(self, coro):
        task = Task(coro, loop=self)
        self._ready.append(task)
        return task
```

## 实际观察 loop 的工作

In [ ]:
import asyncio
import time

async def monitor_loop():
    loop = asyncio.get_running_loop()
    print(f"事件循环类型: {type(loop)}")
    print(f"事件循环 ID: {id(loop)}")
    
    # 观察任务调度
    for i in range(3):
        print(f"迭代 {i}: 时间 {time.time():.2f}")
        await asyncio.sleep(0.5)

# 运行并观察
asyncio.run(monitor_loop())

## 为什么需要事件循环

1. **单线程并发**：在单个线程中实现并发，避免线程切换开销
2. **统一调度**：统一管理所有异步操作
3. **非阻塞 I/O**：高效处理大量 I/O 操作
4. **资源管理**：统一管理定时器、回调函数等资源

## 线程与事件循环的关系

In [ ]:
# Threading部分没仔细研究，这块代码暂时看不懂

import asyncio
import threading

def show_loop_info():
    try:
        loop = asyncio.get_running_loop()
        print(f"线程 {threading.current_thread().name}: 有运行中的事件循环")
    except RuntimeError:
        print(f"线程 {threading.current_thread().name}: 没有运行中的事件循环")

# 主线程
show_loop_info()

# 在事件循环中
async def main():
    show_loop_info()
    
    # 创建新线程
    thread = threading.Thread(target=show_loop_info)
    thread.start()
    thread.join()

asyncio.run(main())

**总结**：`loop` 是一个事件驱动的任务调度器，它在单个线程中管理和执行所有异步操作。它就像一个"交通指挥员"，协调各种异步任务的执行顺序和时机。